In [ ]:
import os
import glob
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
import re
import open3d as o3d
from pycocotools.coco import COCO
import json
import pandas as pd 

# Predict

In [ ]:
import detectron2
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog

In [ ]:
def mask_to_contour(mask):
    contours, _ = cv2.findContours(
        mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
    )
    return contours

def get_random_color():
    return ["#" + "".join([random.choice("ABCDEF0123456789") for _ in range(6)])][
        0
    ].lower()

class dotdict(dict):
    """dot.notation access to dictionary attributes"""

    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, r"", r"") #paths to json and images
register_coco_instances("my_dataset_test", {}, r"", r"") #paths to json and images

In [ ]:
#Metadata: The Metadata instance associated with this name,
#or create an empty one if none is available.
train_metadata = MetadataCatalog.get("my_dataset_train")

#list[dict]: dataset annotations.0
train_dict = DatasetCatalog.get("my_dataset_train")

In [ ]:
from detectron2.engine import DefaultTrainer

from detectron2.evaluation.coco_evaluation import COCOEvaluator
import os
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml")) #choose backbone
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_test",)

cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001

cfg.SOLVER.WARMUP_ITERS = 200
cfg.SOLVER.MAX_ITER = 500 
cfg.SOLVER.GAMMA = 0.05
#cfg.DATASETS.TEST = ()
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.80

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 

cfg.TEST.EVAL_PERIOD = 500

#cfg.PYTORCH_CUDA_ALLOC_CONF

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)


In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(r"") #path to trained model
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.80
predictor = DefaultPredictor(cfg)

In [ ]:
im_path = r"" #path to images 

overlap = 100 #set overlap and image size
size = 512

In [ ]:
numbers = re.compile(r'(\d+)')

def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [ ]:
CATEGORY_TEMPLATE = {
    "id": 1,
    "name": "person",
    "supercategory": None,
    "metadata": {},
    "color": "#69db90",
}

IMAGES_TEMPLATE = {
    "id": 0,
    "width": 530,
    "height": 301,
    "file_name": "",
    "path": "",
    "license": None,
    "fickr_url": None,
    "coco_url": None,
    "date_captured": None,
    "metadata": {},
}

ANNOTATIONS_TEMPLATE = {
    "id": 1,
    "image_id": 0,
    "category_id": 1,
    "width": 530,
    "height": 301,
    "area": 59767,
    "segmentation": [[221, 7, 220, 8, 215, 8]],
    "bbox": [150, 7, 253, 286],
    "metadata": {},
    "color": "#bc07ae",
    "iscrowd": 0,
}

In [ ]:
#this function was for filtering predictions by texture features, I found it did not improve results

def get_pred(imPath, mask):
    
    contours_filtered = []
    
    im = cv2.imread(imPath,cv2.IMREAD_GRAYSCALE)
    
    im_masked = (mask*im).clip(0,255).astype(np.uint8)
    #plt.imshow(im_masked)
    
    #shape = mask.shape
    #mask3 = np.empty([shape[0],shape[1],3])

    #mask3[:,:,0] = mask
    #mask3[:,:,1] = mask
    #mask3[:,:,2] = mask

    #im_masked = (mask3*im).clip(0,255).astype(np.uint8)
    #plt.imshow(im_masked)
    out_list = []
    
    mask = mask.astype(np.uint8)
    contours = mask_to_contour(mask)
    
    #if(len(contours)>0):
    #    for contour in contours:
    #        X,Y,W,H = cv2.boundingRect(contour)
    #        predicted = im_masked[Y:Y+H, X:X+W]
    #        desc = LocalBinaryPatterns(12, 4)
    #        hist, bins, lbp = desc.describe(predicted)
    #        hist = hist.flatten()
    #        plt.imshow(predicted)
    #        out = hist 
    #        out = np.ndarray.tolist(out)
    #        out_list.append(out)
        
    #    data = pd.DataFrame(out_list, columns = ["lbp","lbp.1","lbp.2","lbp.3","lbp.4","lbp.5","lbp.6","lbp.7","lbp.8","lbp.9","lbp.10","lbp.11","lbp.12","lbp.13"])
    #    values = model.predict(data)
    
    #    for x in range(len(contours)):
    #        if (values[x] == 1):
    #            contours_filtered.append(contours[x])
    
    return contours

In [ ]:
def get_anns(contours_filtered):
    
    global IM_NUM
    global ANN_NUM
    
    annotations = []
    
    for contour in contours_filtered:
        
        bbox = cv2.boundingRect(contour)
        segmentation = np.array(contour).flatten().tolist()
        
        
        if (len(segmentation) == 4):
            segmentation.append(segmentation[-2])
            segmentation.append(segmentation[-2])
            print(segmentation)
        
        if (len(segmentation) < 4):
            segmentation.append(segmentation[-2])
            segmentation.append(segmentation[-2])
            segmentation.append(segmentation[-2])
            segmentation.append(segmentation[-2])
            print(segmentation)
            
        
        annotation = dotdict(ANNOTATIONS_TEMPLATE)
        annotation.id = ANN_NUM
        annotation.image_id = IM_NUM
        annotation.category_id = 1
        annotation.width = bbox[2]
        annotation.height = bbox[3]
        annotation.area = cv2.contourArea(contour)
        annotation.segmentation = [segmentation]
        annotation.bbox = [bbox[0],bbox[1],bbox[2],bbox[3]]
        annotation.color = get_random_color()
        
        annotations += [annotation]
        
        ANN_NUM = ANN_NUM + 1
        
    return  annotations

# Predict on one image

In [ ]:
def predict1im(imagePath):
    img = cv2.imread(imagePath)

    height, width, _ = img.shape
    height_new = math.ceil(height/size) * size
    height_scale = height_new/height 
    delta_y= height - height_new
    
    width_new = math.ceil(width/size) * size
    width_scale = width_new/ width
    delta_x = width - width_new
    
    resized_im = cv2.resize(img, (width_new, height_new), cv2.INTER_LINEAR)
    
    num_tiles_i = height_new/size
    num_tiles_j = width_new/size
        #print("num_tiles_i:", num_tiles_i)
        #print("num_tiles_j:", num_tiles_j)
    
    total_tiles = num_tiles_i * num_tiles_j
    
    tile_stack = np.zeros((height_new, width_new, int(total_tiles)))
    
    tile_num = 0
    
    for i in range(height_new//size):
        for j in range(width_new//size):
            
            if(i<num_tiles_i-1):
                overlap_i = overlap
            else:
                overlap_i = 0
            
            if(j<num_tiles_j-1):
                overlap_j = overlap
            else:
                overlap_j = 0
            
            tile = resized_im[i*(size) : (i+1)*size + overlap_i, j*size : (j+1)*size + overlap_j, :]
            #plt.imshow(tile)
            
            outputs = predictor(tile)
            
            v = Visualizer(tile[:, :, ::-1],
                    metadata=train_metadata, 
                    scale=0.5, 
            )
            
            
            instances = outputs["instances"]
            
            pred_masks =  instances.pred_masks
            mask = np.full((pred_masks.shape[1], pred_masks.shape[2]), False)
            #print(mask)
            
            for m in pred_masks:
                mask = mask | m.cpu().numpy()
                
            mask = mask.astype(np.uint8)*255
            
            
            tile_stack[i*(size) : (i+1)*size + overlap_i, j*size : (j+1)*size + overlap_j, tile_num] = mask
            #print(tile_num)
            #print("overlap i:", overlap_i)
            #print("overlap j:", overlap_j)
            
            #print("i:", i)
            #print("j:", j)
    
            tile_num = tile_num + 1 
    
    
    image_pred = np.sum(tile_stack,axis=2)
    image_pred = image_pred.clip(min = 0, max = 255)
    image_pred_resized = cv2.resize(image_pred, (width, height), cv2.INTER_LINEAR) #resize it back down so dimensions will match  
    
    filtered_contours = get_pred(imagePath, image_pred_resized)

    cv2.drawContours(img, filtered_contours, -1, (0,255,0), 10)
    return img

In [ ]:
im_path = r"" #path to images
imPath = os.path.join(im_path,'') #image name

im1 = predict1im(imPath)
plt.imshow(im1)

In [ ]:
im_path = r"" #path to images
imPath2 = os.path.join(im_path,'') #image name

im2 = predict1im(imPath2)
plt.imshow(im2)

In [ ]:
image = cv2.imread(imPath)
plt.imshow(image)

In [ ]:
outputs = predictor(cv2.imread(imPath))  
v = Visualizer(im1[:, :, ::-1],metadata=train_metadata, scale=0.5)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

plt.imshow(out.get_image()[:, :, ::-1])

# Predict on all images

In [ ]:
categories = []

category = dotdict(CATEGORY_TEMPLATE)
category.id = 1
category.name = 'root'
category.color = get_random_color()

categories += [category]

annotations = []
images = []
IM_NUM = 1
ANN_NUM = 1 


for imPath in sorted(glob.glob(im_path + '/*.png'), key = numericalSort):
     
    #plt.clf()
    
    #print("imPath:",imPath)
    
    basename = os.path.basename(imPath)
    
    #print("basename:",basename)
    
    filename = basename.split(".")[0]
    
    #print("filename:",filename)
    
    img = cv2.imread(imPath)
    height, width, _ = img.shape
    height_new = math.ceil(height/size) * size
    height_scale = height_new/height 
    delta_y= height - height_new
    
    width_new = math.ceil(width/size) * size
    width_scale = width_new/ width
    delta_x = width - width_new
    
    resized_im = cv2.resize(img, (width_new, height_new), cv2.INTER_LINEAR)
    
    num_tiles_i = height_new/size
    num_tiles_j = width_new/size
    #print("num_tiles_i:", num_tiles_i)
    #print("num_tiles_j:", num_tiles_j)
    
    total_tiles = num_tiles_i * num_tiles_j
    
    tile_stack = np.zeros((height_new, width_new, int(total_tiles)))
    
    tile_num = 0
    
    for i in range(height_new//size):
        for j in range(width_new//size):
            
            if(i<num_tiles_i-1):
                overlap_i = overlap
            else:
                overlap_i = 0
            
            if(j<num_tiles_j-1):
                overlap_j = overlap
            else:
                overlap_j = 0
            
            tile = resized_im[i*(size) : (i+1)*size + overlap_i, j*size : (j+1)*size + overlap_j, :]
            #plt.imshow(tile)
            
            outputs = predictor(tile)
            
            v = Visualizer(tile[:, :, ::-1],
                   metadata=train_metadata, 
                   scale=0.5, 
            )
            
            
            instances = outputs["instances"]
            
            pred_masks =  instances.pred_masks
            mask = np.full((pred_masks.shape[1], pred_masks.shape[2]), False)
            #print(mask)
            
            for m in pred_masks:
                mask = mask | m.cpu().numpy()
                
            mask = mask.astype(np.uint8)*255
            
            
            tile_stack[i*(size) : (i+1)*size + overlap_i, j*size : (j+1)*size + overlap_j, tile_num] = mask
            #print(tile_num)
            #print("overlap i:", overlap_i)
            #print("overlap j:", overlap_j)
            
            #print("i:", i)
            #print("j:", j)
    
            tile_num = tile_num + 1 
    
    
    image_pred = np.sum(tile_stack,axis=2)
    image_pred = image_pred.clip(min = 0, max = 255)
    image_pred_resized = cv2.resize(image_pred, (width, height), cv2.INTER_LINEAR) #resize it back down so dimensions will match
    
    #print(image_pred_resized.shape)
    
    filtered_contours = get_pred(imPath, image_pred_resized)
    
    image = dotdict(IMAGES_TEMPLATE)
    image.width = image_pred_resized.shape[1]
    image.height = image_pred_resized.shape[0]
    image.file_name = basename
    image.id = IM_NUM
    images += [image]
    
    if(len(filtered_contours)>0):
        anns = get_anns(filtered_contours)
        annotations += anns
    
    IM_NUM = IM_NUM + 1
    #print(IM_NUM) #print the image number if you want to keep track of progress
       

In [ ]:
coco = {"annotations": annotations, "categories": categories, "images": images}

In [ ]:
with open(r"", "w") as outfile: #path to predictions json
    json.dump(coco, outfile)
outfile.close()